### **6.10 - All you need to know about Sensors**

Entre la tonelada de funciones que trae Airflow, hay una que realmente me encanta, que son los "Sensors". ¿Qué es el sensor? El sensor no es más que un operador que espera a que ocurra algo, a que se cumpla una condición antes de pasar a la siguiente tarea. Por ejemplo, quieres esperar a que un archivo aterrice en una ubicación específica, entonces puedes usar el FileSensor, o estás esperando una entrada en tu S3 bucket, entonces puedes usar el S3KeySensor. Así que, básicamente, un sensor no es más que un operador esperando a que una condición sea cierta antes de pasar a la siguiente tarea. Pero el sensor es bastante simple de usar, pero muy potente y hay algunos detalles que necesitas saber sobre él. Vamos a descubrirlos. Digamos que tienes un DAG que está programado (scheduled) para ejecutarse a la 1:00 AM y por algunas razones quieres retrasar la ejecución de una tarea hasta que la hora actual (current datetime) sea las 10:00 AM. ¿Cómo puedes hacerlo sin tener que crear dos DAGs diferentes? Bueno, aquí es donde mediante el uso de un sensor, se puede lograr esto, y más específicamente, es necesario importar el "DatetimeSensor". El objetivo del "DatetimeSensor" es esperar una hora (datetime) determinada antes de seguir adelante. Así que el primer paso es importar ese sensor, y para ello, tienes que escribir "from airflow.sensors.date_time import DateTimesSensor":

<center><img src="https://i.postimg.cc/3rqH0qdD/a1454.png"></center>

y una vez que lo tengas, estarás listo para implementar tu tarea. Ahora, vamos a crear una nueva tarea, y vamos a llamarla "delay". En el caso del DateTimeSensor, sólo hay un argumento específico, que es "target_time" que espera un objeto timedelta. Aquí puedes aprovechar el template engine para decir que quieres esperar hasta la fecha de ejecución actual (current execution date) a las 10:00 AM, y para ello sólo tienes que escribir 2 pares de llaves, luego usar la "execution_date" actual punto "add(hours)" igual a "9". Ok, así que esto es realmente específico para el DateTimeSensor. 

<center><img src="https://i.postimg.cc/BQ1fTCK7/a1455.png"></center>

Ahora, hay otros argumentos que puedes usar en todos tus sensores, y el primero es "poke_interval", que está establecido en 60 segundos por defecto. Y este argumento define la frecuencia con la que su sensor comprobará si su condición es verdadera o no. Así que aquí está verificando cada 60 segundos si la hora actual (current datetime) es a las 10:00 AM o no.

<center><img src="https://i.postimg.cc/4xkkBhvf/a1456.png"></center>

Obviamente puedes aumentar o disminuir ese número, pero como mejor práctica siempre define un valor significativo para el "poke_interval". En ese caso, como estas esperando que la hora actual (current datetime) sea a las 10:00 AM, entonces tiene sentido decir, que quieres verificar cada hora. 

<center><img src="https://i.postimg.cc/K8PXVvQn/a1457.png"></center>

Otro argumento que es extremadamente útil es "mode". Y por defecto está definido como "poke". 

<center><img src="https://i.postimg.cc/SKm0pFtH/a1458.png"></center>

¿Cuál es el objetivo de "mode"? Bueno, en primer lugar, tienes que saber que cada vez que una tarea se está ejecutando en airflow, entonces se toma un worker slot del "default pool" hasta que se complete esa tarea, y como sabes, hay un número limitado de workers slots, y por lo tanto un número de tareas que se pueden ejecutar al mismo tiempo. Pero, con el sensor, estás esperando algo, pero si ese algo nunca sucede, entonces esperarás para siempre y en realidad la mayor parte del tiempo el sensor no hace nada, ¿verdad? Se limita a esperar. Así que estás desperdiciando tus recursos y tu tiempo y otras tareas tendrán que esperar hasta que algunos worker slot sean liberados por tus sensores, es decir, estás desperdiciando un montón de recursos aquí. Así que por eso siempre que estés esperando que el sensor esté tardando más de, digamos 5 minutos o 10 minutos, podría ser útil para ti definir ese "mode" a "reschedule", y con ese modo cada hora, en ese caso, el sensor comprobará si la condición es cierta o no, si no, el worker slot será liberado para que otra tarea pueda ser activada, y 60 minutos más tarde, el sensor tomará un worker slot de nuevo y comprobará la condición. 

<center><img src="https://i.postimg.cc/QMm2842C/a1459.png"></center>

De nuevo, estás optimizando mejor tus recursos cuando usas este "mode", porque, el sensor no tomará un worker slot hasta que se complete. Y durante ese tiempo no hace nada y luego espera a la próxima vez para comprobar si su condición es verdadera o no, bueno, otras tareas podrán ser activadas. Dicho esto, hay otro argumento que usted necesita saber, que se llama "timeout". Y este argumento se establece en "seven days" por defecto. Así que aquí está la cosa, ¿qué pasa si su condición nunca es verdadera? Bueno, su sensor esperará hasta que se agote el tiempo después de siete días. Eso es lo que significa. Y esto es realmente un gran problema para usted, porque, si usted tiene un montón de sensores a la espera de cosas diferentes, o tal vez la misma cosa, pero su condición nunca es verdadera, y en algún momento usted podría terminar con un estancamiento, usted podría terminar en un caso en el que no son capaces de ejecutar más tareas en toda su instancia de Airflow, porque, todos sus worker slots serán tomados por sus sensores. Es por eso que siempre hay que definir un "timeout", un "timeout" significativo. En ese caso, como usted está esperando que la hora de la fecha actual (current datetime) debe ser a las 10:00 AM, en algún momento, entonces usted puede decir que estos sensores deben "timeout" tan pronto como se está ejecutando durante más de 10 horas. Así que usted puede especificar "60 * 60 * 10". De nuevo, te aconsejo encarecidamente que defina siempre un "timeout", de lo contrario, podría acabar con un montón de sensores ocupando un montón de worker slots, para nada. 

<center><img src="https://i.postimg.cc/rmpvKGTJ/a1460.png"></center>

Y no se confunda con el otro argumento que está disponible para cualquier operador, que es "execution_timeout", y este no tiene valor por defecto. Entonces, ¿cuál es la diferencia entre "timeout", que es específico de los sensores y "execution_timeout", que en realidad está disponible para todos los operadores. Bueno, con el "timeout" aquí, 

<center><img src="https://i.postimg.cc/tTm8P1CH/a1461.png"></center>

hay otro argumento que se puede utilizar en combinación, que es "soft_fail". Y cuando se establece "soft_fail" a "True", tan pronto como su sensor se agote (times out), de acuerdo con este timeout (tiempo de espera), 

<center><img src="https://i.postimg.cc/J0DvQ1GH/a1462.png"></center>

entonces su sensor no fallará, pero será omitido. Ese es el propósito del timeout aquí, con "soft_fail". El "execution_timeout", que está disponible para cualquier operador, no tiene en cuenta "soft_fail". Esta es la diferencia. Con "timeout" aquí, somos capaces de decir, si se agota el tiempo (times out), no quiero decir que este sensor ha fallado, quiero ponerlo en el estado "skipped". Finalmente, el último argumento que puede ser útil para usted es "exponential_backoff". Y este cuando se establece en "True", aumentará el tiempo de espera entre cada intervalo de tiempo, entre cada "poke interval". Así, en lugar de esperar 60 segundos, por ejemplo, en el siguiente intervalo de tiempo, usted esperará tal vez 70 segundos y así sucesivamente. ¿Por qué esto puede ser útil para usted? Bueno, si usted está, por ejemplo, utilizando el HTTPSensor y comprobar si una API está disponible o no, usted no desea enviar una tonelada de solicitudes a la misma. Así que, en ese caso, "exponential_backoff" podría ser una buena cosa a utilizar. Ahora ya sabes todo lo que necesitas sobre los sensores y no olvides siempre, siempre definir el timeout.

<center><img src="https://i.postimg.cc/fyGF2692/a1463.png"></center>